In [1]:
authors = {
  "Дарья Донцова":  29369,
  "Джеймс Роллинс": 29442,
  "Макс Фрай":      102994,
  "Эрин Хантер":    26149,
  "Дмитрий Емец":   35952
}

In [2]:
from selenium import webdriver
from time import sleep
import numpy as np
import pandas as pd

Этап 1,получение всех карточек книг.

In [31]:
SCROLL_PAUSE_TIME = 1

In [32]:
ids = []
with webdriver.Firefox() as driver:
    for _, author_id in authors.items():
        driver.get(f'https://www.bookvoed.ru/author/books?id={author_id}')
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            sleep(SCROLL_PAUSE_TIME)
            try:
                driver.find_element_by_class_name("wy").click()
                sleep(SCROLL_PAUSE_TIME)
                print("clicked")
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            except Exception:
                pass
            # Wait to load page
            sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        items = driver.find_elements_by_class_name("gf")
        for item in items:
            ids.append(item.get_attribute("data-book"))
        sleep(SCROLL_PAUSE_TIME)
    
        

clicked
clicked
clicked
clicked
clicked
clicked
clicked
clicked


In [33]:
len(ids)

1770

Сохраним id-шники на всякий случай на диск.Мало ли:

In [38]:
book_ids = pd.DataFrame({'ids': np.array(ids)})
with open('book_ids.csv', mode='w', encoding='utf-8') as f_csv:
    book_ids.to_csv(f_csv, index=False)

Этап 2,получение информации о книгах

Сразу оставлю несколько  комментариев:
1)multiprocessing.Pool у меня постоянно падал,причем я пытался запускать его и на довольно небольших данных,в связи с чем мне пришлось делать на multiprocessing.dummu.Pool.Скорость,конечно,меньше,но все равно быстрее,чем в однопоточном режиме
2)Я решил разбить загрузку на 3 части с сохранением на диск,ибо если запускать всё вместе,в один момент у меня заканчивалась оперативная память и всё было очень плохо
3)Дико извиняюсь за большой вывод в загрузке первой части,у меня,видимо,что-то пошло не так и настройки сбились.


In [3]:
book_ids = pd.read_csv('book_ids.csv')

In [4]:
ids = list(book_ids['ids'])

In [5]:
from multiprocessing import Pool, Value
from multiprocessing.dummy import Pool as ThreadPool
from bs4 import BeautifulSoup
import requests 
import re

In [6]:
ages = {
    'ov': '0+',
    'pv': '6+',
    'qv': '12+',
    'rv': '16+',
    'sv': '18+'
}

In [7]:
str_fields = set(['Код', 'Год', 'Страниц', 'Тираж', 'Понравилось', 'Не понравилось', 'В закладки'])

In [8]:
def get_book_info(book_id):
    global counter
    try:
        res = {"ID": book_id}
        r = requests.get(f'https://www.bookvoed.ru/book?id={book_id}')
        soup = BeautifulSoup(r.text, 'html.parser')
        try:
            res['Обложка'] = soup.find('a', class_='sf ee').attrs['data-fancybox-href']
        except Exception:
            res['Обложка'] = soup.find('a', class_="sf").find('img').attrs['src']
        res['Название'] = soup.find('h1').contents[0]
        res['Рейтинг'] = soup.find("div", class_="af").attrs['style'].strip('width: ').rstrip('%')
        res['Понравилось'] = soup.find("div", class_="y r Ie Ae").find('a').text.strip()
        res['Не понравилось'] = soup.find("div", class_="y s Ie Be").find('a').text.strip()
        res['В закладки'] = soup.find("div", class_="y r Ie ye").find('a').text.strip()
        
        if soup.find("div", class_="lw"):
            res['Описание'] = soup.find("div", class_="lw").text.strip()
        res['Возраст'] = ages[soup.find('div', class_='nM').attrs['class'][0]]
        
        if soup.find('div', class_='Hu Wu'):
            res['Цена'] = float(re.sub("[^0-9]", "",
                soup.find('div', class_='Hu Wu').text.strip()))
        
        for k, v in map(lambda x: x.text.split(":", 1), soup.find_all("tr", class_="uw")):
            res[k] = v

        for key in str_fields:
            if key in res and len(res[key]) > 0:
                res[key] = int(res[key])
        
        if 'Рейтинг' in res and len(res['Рейтинг']) > 0:
            res['Рейтинг'] = float(res['Рейтинг'])
    except Exception:
        print(book_id)
        raise Exception
    print(f'book downloaded! id:{book_id}, num:{counter.value}')
    with counter.get_lock():
        counter.value += 1
    return res

In [9]:
ids_1 = ids[: 900]
ids_2 = ids[900 :]

In [11]:
%%time

counter = None

def init(args):
    ''' store the counter for later use '''
    global counter
    counter = args

counter = Value('i', 0)
pool = ThreadPool(processes=2)
res = pool.map(get_book_info, ids_1)
pool.close()
pool.join()
    
    

book downloaded! id:6799889, num:0
book downloaded! id:9340995, num:1
book downloaded! id:6708508, num:2
book downloaded! id:9340993, num:3
book downloaded! id:6666425, num:4
book downloaded! id:8645468, num:5
book downloaded! id:8628478, num:6
book downloaded! id:8867666, num:7
book downloaded! id:7951791, num:8
book downloaded! id:7493745, num:9
book downloaded! id:7531303, num:10
book downloaded! id:7891402, num:11
book downloaded! id:7528191, num:12
book downloaded! id:8550136, num:13
book downloaded! id:7406599, num:14
book downloaded! id:8626310, num:15
book downloaded! id:7261448, num:16
book downloaded! id:9293590, num:17
book downloaded! id:6799801, num:18
book downloaded! id:9409235, num:19
book downloaded! id:6722263, num:20
book downloaded! id:8626305, num:21
book downloaded! id:6502956, num:22
book downloaded! id:9340994, num:23
book downloaded! id:6077809, num:24
book downloaded! id:8835699, num:25
book downloaded! id:404402, num:26
book downloaded! id:7687613, num:27
boo

book downloaded! id:6895168, num:225
book downloaded! id:8986973, num:226
book downloaded! id:8885768, num:227
book downloaded! id:8985230, num:228
book downloaded! id:8885208, num:229
book downloaded! id:8984466, num:230
book downloaded! id:8884888, num:231
book downloaded! id:8983092, num:232book downloaded! id:8883629, num:232

book downloaded! id:8982250, num:234book downloaded! id:8881186, num:234

book downloaded! id:8880292, num:236book downloaded! id:8979906, num:236

book downloaded! id:8880033, num:238book downloaded! id:8979083, num:238

book downloaded! id:8977974, num:240
book downloaded! id:8879966, num:241
book downloaded! id:8975602, num:242
book downloaded! id:8879041, num:243
book downloaded! id:8975248, num:244
book downloaded! id:8878941, num:245
book downloaded! id:8877869, num:246book downloaded! id:8974252, num:246

book downloaded! id:8974103, num:248book downloaded! id:8876767, num:248

book downloaded! id:8874569, num:250book downloaded! id:8973271, num:250

b

book downloaded! id:8886425, num:448
book downloaded! id:6740792, num:449
book downloaded! id:8885878, num:450
book downloaded! id:6738708, num:451
book downloaded! id:6722118, num:452
book downloaded! id:5883698, num:453
book downloaded! id:6717885, num:454
book downloaded! id:5883248, num:455
book downloaded! id:6716216, num:456
book downloaded! id:5883221, num:457
book downloaded! id:6715979, num:458
book downloaded! id:5846426, num:459
book downloaded! id:6715926, num:460
book downloaded! id:5846374, num:461
book downloaded! id:5842383, num:462
book downloaded! id:6703997, num:463
book downloaded! id:5842317, num:464
book downloaded! id:6699461, num:465
book downloaded! id:5800627, num:466
book downloaded! id:6695889, num:467
book downloaded! id:5798959, num:468
book downloaded! id:6695824, num:469
book downloaded! id:5758275, num:470
book downloaded! id:6694586, num:471
book downloaded! id:6689334, num:472book downloaded! id:5756878, num:472

book downloaded! id:5716754, num:474
b

book downloaded! id:4747237, num:670
book downloaded! id:5925935, num:671
book downloaded! id:5925137, num:672book downloaded! id:4746531, num:672

book downloaded! id:5924905, num:674
book downloaded! id:4710353, num:675
book downloaded! id:5884402, num:676
book downloaded! id:4705433, num:677
book downloaded! id:4705426, num:678
book downloaded! id:3312942, num:679
book downloaded! id:4704388, num:680
book downloaded! id:3311019, num:681
book downloaded! id:4704159, num:682
book downloaded! id:3306046, num:683
book downloaded! id:4663198, num:684
book downloaded! id:3302669, num:685
book downloaded! id:4661466, num:686
book downloaded! id:3300783, num:687
book downloaded! id:4619219, num:688
book downloaded! id:3300308, num:689
book downloaded! id:3295375, num:690
book downloaded! id:4584333, num:691
book downloaded! id:3289794, num:692
book downloaded! id:4579020, num:693
book downloaded! id:3288551, num:694
book downloaded! id:4578783, num:695
book downloaded! id:3283540, num:696
b

book downloaded! id:3356755, num:894book downloaded! id:396892, num:894

book downloaded! id:3356107, num:896
book downloaded! id:3356102, num:897
book downloaded! id:3322397, num:898
book downloaded! id:3313681, num:899
CPU times: user 4min 39s, sys: 3.95 s, total: 4min 43s
Wall time: 6min 40s


In [12]:
df = pd.DataFrame(res)
df.sort_values(by=['ID'], inplace=True)

with open('df.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)

In [10]:
%%time

counter = None

def init(args):
    ''' store the counter for later use '''
    global counter
    counter = args

counter = Value('i', 0)
pool = ThreadPool(processes=2)
res = pool.map(get_book_info, ids_2)
pool.close()
pool.join()

book downloaded! id:8624214, num:0
book downloaded! id:395712, num:1
book downloaded! id:8551443, num:2
book downloaded! id:395322, num:3
book downloaded! id:395191, num:4
book downloaded! id:7403827, num:5
book downloaded! id:394929, num:6
book downloaded! id:7352557, num:7
book downloaded! id:394762, num:8
book downloaded! id:7350180, num:9
book downloaded! id:393981, num:10
book downloaded! id:7316803, num:10
book downloaded! id:393963, num:12
book downloaded! id:7305140, num:13
book downloaded! id:393935, num:14
book downloaded! id:7261326, num:15
book downloaded! id:393764, num:16
book downloaded! id:7210251, num:17
book downloaded! id:393176, num:18
book downloaded! id:6898013, num:19
book downloaded! id:392678, num:20
book downloaded! id:6802584, num:21
book downloaded! id:6802054, num:22
book downloaded! id:390189, num:23
book downloaded! id:6748663, num:24
book downloaded! id:390094, num:25
book downloaded! id:387074, num:26
book downloaded! id:6309260, num:27
book downloaded!

book downloaded! id:4410626, num:227
book downloaded! id:4413802, num:228
book downloaded! id:4410256, num:229
book downloaded! id:8784651, num:230
book downloaded! id:4368924, num:231
book downloaded! id:7694777, num:232
book downloaded! id:4329261, num:233
book downloaded! id:7666472, num:234
book downloaded! id:4326804, num:235
book downloaded! id:7524906, num:236
book downloaded! id:4283904, num:237
book downloaded! id:7415634, num:238
book downloaded! id:4199736, num:239
book downloaded! id:4163509, num:240
book downloaded! id:7352432, num:241
book downloaded! id:7146455, num:242
book downloaded! id:4158077, num:243
book downloaded! id:4073772, num:244
book downloaded! id:7146409, num:245
book downloaded! id:7146267, num:246
book downloaded! id:4034357, num:247
book downloaded! id:7100446, num:248
book downloaded! id:3988945, num:248
book downloaded! id:3947782, num:250
book downloaded! id:7100210, num:251
book downloaded! id:3779220, num:252
book downloaded! id:7100119, num:253
b

book downloaded! id:3356454, num:452
book downloaded! id:381505, num:453
book downloaded! id:3319370, num:454
book downloaded! id:378083, num:455
book downloaded! id:3294613, num:456
book downloaded! id:377765, num:457
book downloaded! id:377324, num:458
book downloaded! id:3242572, num:459
book downloaded! id:376017, num:460
book downloaded! id:670608, num:461
book downloaded! id:376016, num:462
book downloaded! id:644624, num:463
book downloaded! id:376015, num:464
book downloaded! id:617397, num:465
book downloaded! id:371089, num:466
book downloaded! id:432013, num:467
book downloaded! id:359342, num:468
book downloaded! id:416017, num:469
book downloaded! id:351963, num:470
book downloaded! id:407171, num:471
book downloaded! id:349600, num:472
book downloaded! id:404471, num:473
book downloaded! id:347215, num:474
book downloaded! id:404292, num:475
book downloaded! id:346446, num:476
book downloaded! id:399998, num:477
book downloaded! id:345523, num:478
book downloaded! id:3999

book downloaded! id:3777311, num:675
book downloaded! id:7689436, num:676
book downloaded! id:3735897, num:677
book downloaded! id:7535430, num:678
book downloaded! id:3735533, num:679
book downloaded! id:7351295, num:680
book downloaded! id:3695330, num:681
book downloaded! id:7300087, num:682
book downloaded! id:3693490, num:683
book downloaded! id:7261375, num:684
book downloaded! id:3659966, num:685book downloaded! id:7090886, num:685

book downloaded! id:3652539, num:687book downloaded! id:6902168, num:687

book downloaded! id:3651550, num:689
book downloaded! id:6836443, num:690
book downloaded! id:3609762, num:691
book downloaded! id:6801285, num:692
book downloaded! id:3567647, num:693
book downloaded! id:6754513, num:694
book downloaded! id:3532494, num:695
book downloaded! id:6515307, num:696
book downloaded! id:3527003, num:697
book downloaded! id:6512016, num:698
book downloaded! id:6444636, num:699book downloaded! id:3487594, num:699

book downloaded! id:3483105, num:701bo

In [11]:
df = pd.DataFrame(res)
df.sort_values(by=['ID'], inplace=True)

with open('df_1.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)

In [12]:
df_1 = pd.read_csv('df.csv')
df_2 = pd.read_csv('df_1.csv')

In [18]:
frames = [df_1, df_2]
result = pd.concat(frames).reset_index()

In [21]:
result.drop('index', axis=1)

,ID,ISBN,Автор,Авторы,В базе,В закладки,Возраст,Год,Издательство,Код,...,Понравилось,Производитель,Размеры,Рейтинг,Серия,Страниц,Тематика,Тираж,Формат,Цена
0,396892,978-5-699-45287-3,Донцова Дарья Аркадьевна,NaN,Э.ИД.Королева без башни,NaN,16+,2010.0,Эксмо,553655,...,7.0,NaN,"13,00 см x 20,00 см x 3,00 см",71.43,Иронический детектив,384.0,Отечественные,250000.0,84х108/32,NaN
1,397292,978-5-699-45581-2,Донцова Дарья Аркадьевна,NaN,Э.ИД(м).Агент 013,NaN,16+,2010.0,Эксмо,557663,...,3.0,NaN,"11,00 см x 16,50 см x 2,00 см",66.67,Иронический детектив,352.0,Отечественные,260000.0,170.00mm x 105.00mm x 20.00mm,NaN
2,397564,978-5-699-45976-6,Донцова Дарья Аркадьевна,NaN,Э.ВсеЦвДет. Желтый: сборник рассказов,1.0,16+,2010.0,Эксмо,562398,...,NaN,NaN,"10,75 см x 16,50 см x 1,70 см",50.00,Все цвета детектива,288.0,Отечественные,6000.0,70х90/32,NaN
3,397671,978-5-699-46122-6,Донцова Дарья Аркадьевна,NaN,Э.ИД.Страстная ночь в зоопарке,NaN,16+,2011.0,Эксмо,563581,...,4.0,NaN,"13,00 см x 20,00 см x 2,60 см",63.64,Иронический детектив,352.0,Отечественные,250000.0,84х108/32,NaN
4,397697,978-5-699-44565-3,Донцова Дарья Аркадьевна,NaN,Э.ИД. Кулинарная книга лентяйки-3: Праздник по...,3.0,12+,2008.0,Эксмо,560078,...,4.0,NaN,"21,50 см x 17,00 см",70.00,Иронический детектив,304.0,"Кулинария, напитки",50000.0,70х90/16,NaN
5,398758,978-5-699-46646-7,Донцова Дарья Аркадьевна,NaN,Э.ИД(м).Приданое лохматой обезьяны,NaN,16+,2011.0,Эксмо,652747,...,3.0,NaN,"11,00 см x 16,50 см x 2,30 см",60.00,Иронический детектив,384.0,Отечественные,250000.0,70х90/32,NaN
6,399453,978-5-699-47981-8,Донцова Дарья Аркадьевна,NaN,Э.ИД.Дедушка на выданье,11.0,16+,2011.0,Эксмо,669216,...,20.0,NaN,"12,00 см x 19,00 см x 2,50 см",90.91,Иронический детектив,352.0,Отечественные,70100.0,80х100/32,NaN
7,399642,978-5-699-47207-9,Донцова Дарья Аркадьевна,NaN,Э.ИДЭ.Лебединое озеро Ихтиандрароман,5.0,16+,2015.0,Эксмо,671138,...,15.0,NaN,"10,75 см x 16,50 см",83.33,Иронический детектив,384.0,Отечественные,250000.0,165.00mm x 107.00mm x 20.00mm,NaN
8,399687,978-5-699-47183-6,Донцова Дарья Аркадьевна,NaN,Э.ВЦД(м).Зеленый,5.0,16+,2011.0,Эксмо,688742,...,1.0,NaN,"10,75 см x 16,50 см",50.00,Все цвета детектива (обложка),352.0,Отечественные,5000.0,70х90/32,NaN
9,400095,978-5-699-48429-4,Донцова Дарья Аркадьевна,NaN,Э.ИД. В постели с Кинг-Конгом: роман,1.0,16+,2011.0,Эксмо,714994,...,7.0,NaN,"12,00 см x 19,00 см",71.43,Иронический детектив,352.0,Отечественные,250000.0,80х100/32,NaN
